In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import collections
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM, TimeDistributed

from sklearn.metrics import roc_auc_score, roc_curve, auc

import matplotlib
# matplotlib.use('Agg')
import matplotlib.pyplot as plt
# import collections

import os
import subprocess
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM, TimeDistributed

from sklearn.metrics import roc_auc_score, roc_curve, auc


def getData(meta, X_raw, tgt, future):
	"""
	Let us say that target 'tgt' is the index of a tile, 
	so we predict for all three variables, CASE
	"""
	
	AR = meta.loc[tgt, 'CASE']
	#PR = meta.loc[tgt, 'BURGLARY-THEFT-MOTOR_VEHICLE_THEFT']
	#AS = meta.loc[tgt, 'HOMICIDE-ASSAULT-BATTERY']
	
	#print('Arrest: {},\nProperty: {},\nAssault: {}.'.format(AR, PR, AS))
	
	X = X_raw.T
	Y = X_raw[[AR]].T

	# get train and test
	# Train is the first two years (2014, 2015), so of length 365 * 2 = 730
	# test is the last year (2016, a leap year), so of length 366
	X_train = X[:30 - future]
	Y_train = Y[future: 30]

	X_test = X[30 - future: -future]
	Y_test = Y[30:]
	
	# Reshape to fit RNN
	# The dimension of RNN input/output is (num_samples, length_in_time, data_dimension)
	X_train = X_train.reshape(1, *X_train.shape)
	Y_train = Y_train.reshape(1, *Y_train.shape)

	X_test = X_test.reshape(1, *X_test.shape)
	Y_test = Y_test.reshape(1, *Y_test.shape)
	
	print('Training data: input dim = {}, output dim = {}'.format(X_train.shape, Y_train.shape))
	print('Out-sample data: input dim = {}, output dim = {}'.format(X_test.shape, Y_test.shape))

	return X_train, Y_train, X_test, Y_test


def train(X_train, Y_train, epochs=200):

	model = tf.keras.Sequential()

	# Don't use unless you are sure test length is the same
	# as the train length, which is not the case for us
	# model.add(tf.keras.Input(X_train.shape[1:]))  

	# Two LSTM layers
	model.add(LSTM(units=30, input_shape=(None, X_train.shape[-1]), return_sequences=True))
	model.add(LSTM(units=10, return_sequences=True))
	
	# One output layers
	model.add(TimeDistributed(Dense(units=1, activation='sigmoid')))
	
	model.compile(loss='mse', optimizer='adam')

	model.summary()

	model.fit(X_train, Y_train, epochs=epochs, batch_size=1, verbose=1)
	
	return model


def train_3(X_train, Y_train, epochs=200):

	model = tf.keras.Sequential()

	# Don't use unless you are sure test length is the same
	# as the train length, which is not the case for us
	# model.add(tf.keras.Input(X_train.shape[1:]))  

	# Two LSTM layers
	model.add(LSTM(units=50, input_shape=(None, X_train.shape[-1]), return_sequences=True))
	model.add(LSTM(units=10, return_sequences=True))
	model.add(LSTM(units=10, return_sequences=True))
	
	# One output layers
	model.add(TimeDistributed(Dense(units=1, activation='sigmoid')))
	
	model.compile(loss='mse', optimizer='adam')

	model.summary()

	model.fit(X_train, Y_train, epochs=epochs, batch_size=1, verbose=1)
	
	return model



In [3]:
#meta = pd.read_csv('meta.csv', index_col=0)
meta = pd.read_csv('meta2.csv', index_col=0)
#X_raw = np.genfromtxt('CRIME-_2014-01-01_2016-12-31.csv')
X_raw = np.genfromtxt('/home/ishanu/Dropbox/ZED/Research/CCTS40500_/notebooks/time_series_covid/data.csv')


In [4]:
meta

,CASE,lat1,lat1.1,lon1,lon2
31.82571#117.2264#CASE,1,31.82571,31.82571,117.2264,117.2264
40.18238#116.4142#CASE,2,40.18238,40.18238,116.4142,116.4142
30.05718#107.874#CASE,3,30.05718,30.05718,107.8740,107.8740
26.07783#117.9895#CASE,4,26.07783,26.07783,117.9895,117.9895
36.0611#103.8343#CASE,5,36.06110,36.06110,103.8343,103.8343
...,...,...,...,...,...
50.5039#4.4699#CASE,70,50.50390,50.50390,4.4699,4.4699
43.0731#-89.4012#CASE,71,43.07310,43.07310,-89.4012,-89.4012
35.4437#129.638#CASE,72,35.44370,35.44370,129.6380,129.6380
32.7157#-117.1611#CASE,73,32.71570,32.71570,-117.1611,-117.1611


In [5]:
cynetTop = meta.index.values

In [6]:
X_raw.shape

(74, 45)

## An example

In [10]:
center = cynetTop[0]
tile = meta.loc[center, ['lat1', 'lat2','lon1', 'lon2']]
    
future = 20
    
X_train, Y_train, X_test, Y_test = getData(meta, X_raw, center, future)

model = train_3(X_train, Y_train, epochs=20)
prediction = model.predict(X_test)

W0308 15:47:26.587787 140026871625216 deprecation.py:506] From /home/ishanu/.local/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Training data: input dim = (1, 10, 74), output dim = (1, 10, 1)
Out-sample data: input dim = (1, 15, 74), output dim = (1, 15, 1)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, None, 50)          25000     
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 10)          2440      
_________________________________________________________________
lstm_2 (LSTM)                (None, None, 10)          840       
_________________________________________________________________
time_distributed (TimeDistri (None, None, 1)           11        
Total params: 28,291
Trainable params: 28,291
Non-trainable params: 0
_________________________________________________________________


W0308 15:47:29.384397 140026871625216 deprecation.py:323] From /home/ishanu/.local/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/20
1/1 [==============================] - 4s 4s/sample - loss: 51283.7188
Epoch 2/20
1/1 [==============================] - 0s 35ms/sample - loss: 51280.2500
Epoch 3/20
1/1 [==============================] - 0s 134ms/sample - loss: 51276.7227
Epoch 4/20
1/1 [==============================] - 0s 139ms/sample - loss: 51273.3125
Epoch 5/20
1/1 [==============================] - 0s 121ms/sample - loss: 51270.2266
Epoch 6/20
1/1 [==============================] - 0s 102ms/sample - loss: 51267.3828
Epoch 7/20
1/1 [==============================] - 0s 58ms/sample - loss: 51264.6797
Epoch 8/20
1/1 [==============================] - 0s 42ms/sample - loss: 51261.9922
Epoch 9/20
1/1 [==============================] - 0s 125ms/sample - loss: 51259.3516
Epoch 10/20
1/1 [==============================] - 0s 93ms/sample - loss: 51256.6562
Epoch 11/20
1/1 [==============================] - 0s 130ms/sample - loss: 51253.9648
Epoch 12/20
1/1 [==============================] - 0s 77ms/sample - lo

In [ ]:
figTitle = '#'.join(map(str, tile.values)) + '_{}'.format(future)
dfName = figTitle + '.rnnres'

In [ ]:
Y_test=np.squeeze(Y_test)
prediction=np.squeeze(prediction)

In [ ]:
from sklearn.metrics import roc_auc_score
np.corrcoef(Y_test, prediction)

In [ ]:
Y_test

In [ ]:
prediction